In [7]:
!pip install sagemaker --upgrade --quiet

In [8]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"Sagemaker role arn: {role}")
print(f"Sagemaker session region: {sess.boto_region_name}")

Sagemaker role arn: arn:aws:iam::112330571794:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
Sagemaker session region: eu-west-1


In [18]:
import json
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

instance_type = "ml.g5.4xlarge"

config = {
    'HF_MODEL_ID': "tiiuae/falcon-7b-instruct",
    'SM_NUM_GPUS': json.dumps(1),          # Number of GPU used per replica
    'MAX_INPUT_LENGTH': json.dumps(1024),  # Max length of input text
    'MAX_TOTAL_TOKENS': json.dumps(2048),  # Max length of the generation (including input text)
}

model = HuggingFaceModel(
    role=role,
    image_uri=get_huggingface_llm_image_uri("huggingface"),  # get the newest Huggingface TGI container
    env=config
)

In [19]:
# Deploy model to an endpoint
llm = model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=300,
)

-------------!

In [20]:
llm.delete_model()
llm.delete_endpoint()